In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

# An Investigation into Flight Delays

### The flights dataset

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). There are data for the years 1987 - 2018. See the description of columns in `data/columns.txt`.

This project will look at a single year (2015) to keep the analysis "simple", which is available at the URL below (*NOT* on the data.gov site).


### Creating the datasets

**Step 1**

The flights dataset for 2015 is not small (~600MB). While I could likely load the entire dataset into Pandas on my laptop, if I wanted to work with more than one year, this would quickly become difficult (the data is available for 1987-2018). Therefore, I will filter down the dataset into two smaller files without ever reading the larger dataset fully into memory. We are going to create two smaller datasets:

1. All flights arriving or departing from San Diego International Airport in 2015.
2. All flights flown by either JetBlue or Southwest Airline in 2015.

---

To do this, we are going to use the `chunksize=N` keyword in Pandas `read_csv` to read the flights dataset in blocks of `N` lines. When we use this keyword argument, `pd.read_csv(fp, chunksize=N)` becomes a *iterator* that iterates through dataframes of length N until we have reached the end of the dataset. A typical pattern looks like:
```
L = pd.read_csv(filepath, chunksize=1000)
for df in L:
    process(df)
```
Where each `df` is a dataframe of length 1000. 

The processing we are going to do is:
1. Iterate through the dataset, chunk-by-chunk,
2. Filtering out rows of each chunk
3. Incrementally add to a filtered csv file (since the data is perhaps too big to keep in memory). Keep in mind, if you want to keep writing to the same file, the mode='a' keyword in the `.to_csv` method can be helpful when calling it in the loop (a stands for 'append')

---

Write two functions that create the datasets below, using the 'chunking' pattern described above. Your functions should use `chunksize` of 10000.
1. `get_san` which takes in a filepath containing all flights and an filepath where filtered dataset #1 is written. The function should return `None`.
1. `get_jb_sw` which takes in a filepath containing all flights and an filepath where filtered dataset #2 is written. The function should return `None`.


In [5]:
def process_SAN(df):
    ori = df[df['ORIGIN_AIRPORT'] == 'SAN']
    dest = df[df['DESTINATION_AIRPORT'] == 'SAN']
    return ori.append(dest)

def get_san(infp, outfp):
    """
    get_san takes in a filepath containing all flights and a filepath where
    filtered dataset #1 is written (that is, all flights arriving or departing
    from San Diego International Airport in 2015).
    The function should return None.
    """
    result = pd.DataFrame()
    L = pd.read_csv(infp, chunksize=10000)
    for df in L:
        result = pd.concat([process_SAN(df), result])
    result.to_csv(outfp, index=False)

def process_airlines(df):
    B6 = df[df['AIRLINE'] == 'B6']
    JBU = df[df['AIRLINE'] == 'JBU']
    WN = df[df['AIRLINE'] == 'WN']
    SWA = df[df['AIRLINE'] == 'SWA']
    return B6.append(JBU).append(WN).append(SWA)
    
def get_sw_jb(infp, outfp):
    """
    get_sw_jb takes in a filepath containing all flights and a filepath where
    filtered dataset #2 is written (that is, all flights flown by either
    JetBlue or Southwest Airline in 2015).
    The function should return None.
    """
    result = pd.DataFrame()
    L = pd.read_csv(infp, chunksize=10000)
    for df in L:
        result = pd.concat([process_airlines(df), result])
    result.to_csv(outfp, index=False)

In [6]:
#1
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'santest.tmp')
get_san(infp, outfp)
df = pd.read_csv(outfp)
print(df.shape)
#     (53, 31)
os.remove(outfp)


(53, 31)


In [7]:
#2
infp = os.path.join('data', 'flights.test')
outfp = os.path.join('data', 'jbswtest.tmp')
get_sw_jb(infp, outfp)
df = pd.read_csv(outfp)
print(df.shape)
#     (73, 31)
os.remove(outfp)


(73, 31)


In [8]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,8,21,5,B6,19,N584JB,BOS,SAN,1629,...,2000,27.0,False,False,NaN,0.0,0.0,27.0,0.0,0.0
1,2015,6,24,3,B6,190,N648JB,SAN,JFK,1252,...,2122,0.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,3,3,2,B6,584,N613JB,MCO,JFK,940,...,1211,-2.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,6,2,2,B6,462,N639JB,SJU,BOS,1153,...,1548,-12.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,10,26,1,B6,1386,N216JB,12478,14576,1259,...,1409,-7.0,False,False,NaN,NaN,NaN,NaN,NaN,NaN


# Flight Delays to/from San Diego

The department of transportation has all flight delays for listed years on their [website](https://catalog.data.gov/dataset/airline-on-time-performance-and-causes-of-flight-delays-on-time-data). 

The zip file at the [URL](https://dsc80-fa19-data.s3-us-west-2.amazonaws.com/project02/flight-delays.zip) contains a file `to_from_san.csv` that consists of all flights either to or from SAN (San Diego) in 2015 -- i.e. the output of step 1. This dataset should match the dataset that your code returned in question 1.

Read in `to_from_san.csv` using `read_csv` and inspect the dataframe for an initial assessment about the data quality.

In [9]:
# Run this cell
to_from_san_filepath = os.path.join('data', 'to_from_san.csv')
flights = pd.read_csv(to_from_san_filepath)

In [10]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,DL,978,N693DL,SAN,SLC,615,...,906.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,OO,5608,N930SW,SAN,LAX,615,...,702.0,-5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,WN,823,N7707C,SAN,BWI,620,...,1352.0,-23.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,WN,603,N461WN,SAN,MDW,620,...,1201.0,-29.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,UA,1192,N69804,SAN,DEN,620,...,936.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### Understanding the data types of the columns

**Step 2**:

* First, classify the *kind* of data each column contains. Create a function `data_kinds` of zero variables which outputs a (hard-coded) dictionary of data kinds, keyed by column name, with values `Q`, `O`, `N` (for 'Quantitative', 'Ordinal', or 'Nominal').

* Second, decide the best data *type* for each column. Create a function `data_types` of zero variables which outputs a (hard-coded) dictionary of data types, keyed by column name, with values `str`, `int`, `float`, `bool`. 

*Remark 1*: A column which *should* be `int`s, but contains `NaN`, *must* be a float column. See Lecture 2 notes an explanation of `NaN` and data-types.

*Remark 2*: As with real data, some data processing decisions may be ambiguous here. Make a best decision using the information available to you. It may be helpful to (re)read the relevant [section of the textbook](https://afraenkel.github.io/practical-data-science/03/kinds-of-data.html). 
* Certain answers *may* have more than one correct answer (in these cases, more than one choice gets full credit),
* All answers will be graded for partial credit (some wrong answers are more wrong than other).
There are many columns, so don't worry about the correctness of any given one; do make sure you are thinking about what's contained in a column critically!

In [11]:
def data_kinds():
    """
    data_kinds outputs a (hard-coded) dictionary of data kinds, keyed by column
    name, with values Q, O, N (for 'Quantitative', 'Ordinal', or 'Nominal').

    :Example:
    >>> out = data_kinds()
    >>> isinstance(out, dict)
    True
    >>> set(out.values()) == {'O', 'N', 'Q'}
    True
    """

    return {'YEAR': 'Q', 'MONTH': 'Q', 'DAY': 'Q', 'DAY_OF_WEEK': 'Q', 'AIRLINE': 'N', 'FLIGHT_NUMBER': 'N',
       'TAIL_NUMBER': 'Q', 'ORIGIN_AIRPORT': 'N', 'DESTINATION_AIRPORT': 'N',
       'SCHEDULED_DEPARTURE': 'Q', 'DEPARTURE_TIME': 'Q', 'DEPARTURE_DELAY': 'Q', 'TAXI_OUT': 'Q',
       'WHEELS_OFF': 'Q', 'SCHEDULED_TIME': 'Q', 'ELAPSED_TIME': 'Q', 'AIR_TIME': 'Q', 'DISTANCE': 'Q',
       'WHEELS_ON': 'Q', 'TAXI_IN': 'Q', 'SCHEDULED_ARRIVAL': 'Q', 'ARRIVAL_TIME': 'Q',
       'ARRIVAL_DELAY': 'Q', 'DIVERTED': 'O', 'CANCELLED': 'O', 'CANCELLATION_REASON': 'O',
       'AIR_SYSTEM_DELAY': 'Q', 'SECURITY_DELAY': 'Q', 'AIRLINE_DELAY': 'Q',
       'LATE_AIRCRAFT_DELAY': 'Q', 'WEATHER_DELAY': 'Q'}


def data_types():
    """
    data_types outputs a (hard-coded) dictionary of data types, keyed by column
    name, with values str, int, float.

    :Example:
    >>> out = data_types()
    >>> isinstance(out, dict)
    True
    >>> set(out.values()) == {'int', 'str', 'float', 'bool'}
    True
    """

    return {'YEAR': 'int', 'MONTH': 'int', 'DAY': 'int', 'DAY_OF_WEEK': 'int', 'AIRLINE': 'str', 'FLIGHT_NUMBER': 'int',
       'TAIL_NUMBER': 'str', 'ORIGIN_AIRPORT': 'str', 'DESTINATION_AIRPORT': 'str',
       'SCHEDULED_DEPARTURE': 'int', 'DEPARTURE_TIME': 'float', 'DEPARTURE_DELAY': 'float', 'TAXI_OUT': 'float',
       'WHEELS_OFF': 'float', 'SCHEDULED_TIME': 'int', 'ELAPSED_TIME': 'float', 'AIR_TIME': 'float', 'DISTANCE': 'int',
       'WHEELS_ON': 'float', 'TAXI_IN': 'float', 'SCHEDULED_ARRIVAL': 'int', 'ARRIVAL_TIME': 'float',
       'ARRIVAL_DELAY': 'float', 'DIVERTED': 'bool', 'CANCELLED': 'bool', 'CANCELLATION_REASON': 'str',
       'AIR_SYSTEM_DELAY': 'float', 'SECURITY_DELAY': 'float', 'AIRLINE_DELAY': 'float',
       'LATE_AIRCRAFT_DELAY': 'float', 'WEATHER_DELAY': 'float'}

In [12]:
#1

out = data_kinds()
print(isinstance(out, dict))
#     True
set(out.values()) == {'O', 'N', 'Q'}

True


True

In [13]:
# 2
out = data_types()
print(isinstance(out, dict))
#     True
set(out.values()) == {'int', 'str', 'float', 'bool'}
#     True

True


True

### Read in the typed flights data

Read in the flights data using your dictionary of data-types in `read_csv`. This both speeds up parsing, as well as gives you the correct data-types upon reading (which columns would pandas *parse incorrectly* if you didn't use a `dtype` dictionary?)

In [14]:
# Run this cell
dtypes = proj.data_types()
flights = pd.read_csv(to_from_san_filepath, dtype=dtypes)

**Step 3 (Basic Stats):**

Define a function `basic_stats` that takes `flights` and outputs a dataframe that contains statistics for flights arriving/departing for SAN. That is, the output should have two rows, indexed by `ARRIVING` and `DEPARTING`, and have the following columns:

1. number of arriving/departing flights to/from SAN (`count`).
2. mean flight (arrival) delay of arriving/departing flights to/from SAN (`mean_delay`).
3. median flight (arrival) delay of arriving/departing flights to/from SAN (`median_delay`).
4. the airline code of the airline with the longest flight (arrival) delay among all flights arriving/departing to/from SAN (`airline`).
5. a list of the three months with the greatest number of arriving/departing flights to/from SAN, sorted from greatest to least (`top_months`).



In [15]:
def basic_stats(flights):
    """
    basic_stats takes flights and outputs a dataframe that contains statistics
    for flights arriving/departing for SAN.
    That is, the output should have have two rows, indexed by ARRIVING and
    DEPARTING, and have the following columns:

    * number of arriving/departing flights to/from SAN (count).
    * mean flight (arrival) delay of arriving/departing flights to/from SAN
      (mean_delay).
    * median flight (arrival) delay of arriving/departing flights to/from SAN
      (median_delay).
    * the airline code of the airline with the longest flight (arrival) delay
      among all flights arriving/departing to/from SAN (airline).
    * a list of the three months with the greatest number of arriving/departing
      flights to/from SAN, sorted from greatest to least (top_months).

    """
    departing = flights[flights['ORIGIN_AIRPORT']=='SAN']
    arriving = flights[flights['DESTINATION_AIRPORT']=='SAN']
    arr_count = arriving.shape[0]
    dep_count = departing.shape[0]
    arr_mean = np.nanmean(arriving['ARRIVAL_DELAY'])
    dep_mean = np.nanmean(departing['ARRIVAL_DELAY'])
    arr_median = np.nanmedian(arriving['ARRIVAL_DELAY'])
    dep_median = np.nanmedian(departing['ARRIVAL_DELAY'])
    arr_airling = arriving[arriving['ARRIVAL_DELAY'] == np.max(arriving['ARRIVAL_DELAY'])]['AIRLINE'].values[0]
    dep_airling = departing[departing['ARRIVAL_DELAY'] == np.max(departing['ARRIVAL_DELAY'])]['AIRLINE'].values[0]
    arr_months = arriving.groupby("MONTH", as_index=False).count().sort_values('YEAR', ascending=False)['MONTH'].values[:3].tolist()
    dep_months = departing.groupby("MONTH", as_index=False).count().sort_values('YEAR', ascending=False)['MONTH'].values[:3].tolist()
    data = {'count':[arr_count, dep_count],
            'mean_delay':[arr_mean, dep_mean],
           'median_delay': [arr_median, dep_median],
           'airline': [arr_airling, dep_airling],
           'top_months': [arr_months, dep_months]} 
    df = pd.DataFrame(data)
    df.index = ['ARRIVING', 'DEPARTING']
    return df

In [16]:
>>> fp = os.path.join('data', 'to_from_san.csv')
>>> dtypes = data_types()
>>> flights = pd.read_csv(fp, dtype=dtypes)
>>> out = basic_stats(flights)
>>> print(out.index.tolist() == ['ARRIVING', 'DEPARTING'])
# True
>>> cols = ['count', 'mean_delay', 'median_delay', 'airline', 'top_months']
>>> print(out.columns.tolist() == cols)
# True

True
True


In [117]:
fp = os.path.join('data', 'to_from_san.csv')
dtypes = proj.data_types()
flights = pd.read_csv(fp, dtype=dtypes)
out = proj.basic_stats(flights)
out.index.tolist() == ['ARRIVING', 'DEPARTING']
#     True
cols = ['count', 'mean_delay', 'median_delay', 'airline', 'top_months']
out.columns.tolist() == cols
out

,count,mean_delay,median_delay,airline,top_months
ARRIVING,70207,3.676826,-4.0,AA,"[7, 8, 6]"
DEPARTING,70207,3.328988,-5.0,AA,"[7, 8, 6]"


### Understanding flight delays: Departures, Arrivals, and everything in-between

**Step 4**

Often `DEPARTURE_DELAY` is thought to be the main cause of a flight delay -- i.e., when the flight is late pushing off from the gate. 

However, there are other ways that flights can be late: waiting on the tarmac, headwinds, turbulence, circling a busy airport, and waiting for a gate after landing. First, we will analyze all the ways in which a flight can be delayed.

* First, create a function `depart_arrive_stats` that takes in a dataframe like `flights` and calculates the following quantities in a series, indexed by `late1`, `late2`, `late3`:
    - The proportion of flights from/to SAN that leave late, but arrive early or on-time (`late1`).
    - The proportion of flights from/to SAN that leaves early, or on-time, but arrives late (`late2`).
    - The proportion of flights from/to SAN that both left late and arrived late (`late3`).
    
* Second, create a function `depart_arrive_stats_by_month` that takes in a dataframe like `flights` and calculates the quantities above broken down by *month*. That is, the output is a dataframe, indexed by `MONTH`, with columns given by `late1`, `late2`, `late3`.

*Remark:* Does this question reveal any data quality issues? Can you pinpoint when these issues occur?

In [17]:
def depart_arrive_stats(flights):
    """
    depart_arrive_stats takes in a dataframe like flights and calculates the
    following quantities in a series (with the index in parentheses):
    - The proportion of flights from/to SAN that
      leave late, but arrive early or on-time (late1).
    - The proportion of flights from/to SAN that
      leaves early, or on-time, but arrives late (late2).
    - The proportion of flights from/to SAN that
      both left late and arrived late (late3).
    """
    #late 1
    late1 = ((flights['ARRIVAL_DELAY'] <= 0) & (flights['DEPARTURE_DELAY'] > 0)).mean()
    #late 2
    late2 = ((flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0)).mean()
    #late 3
    late3 = ((flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0)).mean()
    return pd.Series([late1, late2, late3], index=["late1", "late2", "late3"])


def depart_arrive_stats_by_month(flights):
    """
    depart_arrive_stats_by_month takes in a dataframe like flights and
    calculates the quantities in depart_arrive_stats, broken down by month
    """
    
    month = flights[['MONTH', 'DEPARTURE_DELAY', 'ARRIVAL_DELAY']].copy()
    late1 = (month['ARRIVAL_DELAY'] <= 0) & (month['DEPARTURE_DELAY'] > 0)
    late2 = (flights['DEPARTURE_DELAY'] <= 0) & (flights['ARRIVAL_DELAY'] > 0)
    late3 = (flights['DEPARTURE_DELAY'] > 0) & (flights['ARRIVAL_DELAY'] > 0)
    month['late1'] = late1
    month['late2'] = late2
    month['late3'] = late3
    return month.groupby('MONTH')[['late1', 'late2', 'late3']].mean()

In [19]:
out = depart_arrive_stats(flights)
out

late1    0.119853
late2    0.089329
late3    0.278804
dtype: float64

In [18]:
out = depart_arrive_stats_by_month(flights)
out.columns.tolist() == ['late1', 'late2', 'late3']
#     True
print(set(out.index) <= set(range(1, 13)))
# #     True
out

True


,late1,late2,late3
MONTH,,,
1,0.119063,0.095645,0.270501
2,0.118087,0.096915,0.289925
3,0.130521,0.089240,0.258309
4,0.116475,0.095064,0.264326
5,0.114136,0.083498,0.285151
6,0.121138,0.088248,0.330968
7,0.115371,0.095455,0.335799
8,0.123828,0.086890,0.272450
9,0.110409,0.087890,0.185905


### Flight delays and day of the week

**Step 5**

Next, we'd like to understand the flight traffic to/from SAN by day of the week. Day of the week is specified by integers 1 through 7; verify which integer corresponds to which day.

Next create two functions to understand both the amount of traffic and the average flight delay of flights for each airline by day-of-the week. We both want to understand *presence* each airline has as well as their *performance*.

1. Create a function `cnts_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, how many flights were there (in 2015)?


2. Create a function `mean_by_airline_dow` that takes in a dataframe like `flights` and outputs a dataframe that answers the following question: Given any `AIRLINE` and `DAY_OF_WEEK`, what is the average `ARRIVAL_DELAY` (in 2015)?

Both dataframes should have a column for each distinct value of `AIRLINE` and a row for each `DAY_OF_WEEK`.

The output has the *form* of the table below (not the entries themselves!)

<img src="pivot.png" />

In [20]:
def cnts_by_airline_dow(flights):
    """
    mean_by_airline_dow takes in a dataframe like flights and outputs a
    dataframe that answers the question:
    Given any AIRLINE and DAY_OF_WEEK, how many flights were there (in 2015)?
    """
    df = flights.pivot_table(
        values  = "DAY",
        index   = "DAY_OF_WEEK",
        columns = "AIRLINE",
        aggfunc = "count",
        fill_value = 0
    )
    return df    


def mean_by_airline_dow(flights):
    """
    mean_by_airline_dow takes in a dataframe like flights and outputs a
    dataframe that answers the question:
    Given any AIRLINE and DAY_OF_WEEK, what is the average ARRIVAL_DELAY (in
    2015)?
    """
    df2 = flights.pivot_table(
        values  = "ARRIVAL_DELAY",
        index   = "DAY_OF_WEEK",
        columns = "AIRLINE",
        aggfunc = "mean",
        fill_value = 0
    )
    return df2


In [22]:
out = cnts_by_airline_dow(flights)
print(set(out.columns) == set(flights['AIRLINE'].unique()))
#     True
print(set(out.index) == set(flights['DAY_OF_WEEK'].unique()))
#     True
print((out >= 0).all().all())
#     True
out

True
True
True


AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,1915,1564,384,1753,218,96,531,1859,2369,495,536,9238
2,1872,1345,359,1647,210,96,529,1829,2211,463,544,9300
3,1922,1428,359,1719,227,96,530,1814,2293,472,542,9328
4,1888,1551,380,1729,220,96,531,1835,2328,510,531,9108
5,1859,1449,376,1701,208,94,518,1813,2279,505,523,9006
6,1755,1476,270,1433,195,94,518,1837,1743,496,365,7201
7,1886,1465,387,1726,214,96,532,1889,2172,503,443,8587


In [23]:
#2
out = proj.mean_by_airline_dow(flights)
print(set(out.columns) == set(flights['AIRLINE'].unique()))
#     True
print(set(out.index) == set(flights['DAY_OF_WEEK'].unique()))
#     True
out#5.2

True
True


AIRLINE,AA,AS,B6,DL,F9,HA,NK,OO,UA,US,VX,WN
DAY_OF_WEEK,,,,,,,,,,,,
1,2.245981,-2.652286,0.928571,-3.096904,3.233945,5.842105,9.222656,7.405257,3.081280,1.764344,12.804878,6.355997
2,0.791037,-4.535474,4.507123,-2.988408,4.975728,7.385417,10.669903,5.109574,5.617431,1.354626,4.645522,5.394194
3,1.811808,-4.030899,2.735043,-2.437098,5.634361,16.833333,6.304264,5.566741,3.686344,-2.145299,0.631970,3.556762
4,4.076552,-2.180995,3.152406,-2.339721,4.531818,7.552083,7.931298,7.971901,4.957124,1.198397,8.277040,6.582300
5,1.235679,-1.040887,3.329759,-5.699528,4.830918,8.021505,7.242718,5.920156,1.756997,3.017928,9.325536,7.961504
6,-0.427912,-3.283673,-1.854478,-3.608726,0.119171,7.159574,6.943026,3.746961,-1.338924,-2.227926,0.101370,1.684625
7,1.611924,-2.318213,2.422043,-4.857392,6.112150,5.229167,8.754753,8.570806,-0.190121,2.257545,12.594533,6.867543


### Understanding null values in the flights data

**Step 6 (Missing by Design)**

Now we would like to understand how data is missing in the flights data. First, compute the proportion of each column of `flights` which are non-null. 

Recall that a column is *missing by design* if you can determine when the entry of a column is missing based solely on other data in the same row. That is
* there is *no randomness* in determining when an entry is missing.
* you can describe when the column is missing a value with a logical (not random) condition.
* you can express which rows will have missing values in terms of logical statements about the *other* columns in the same row.

For this question, verify the following columns are *missing by design*:
* The column `ARRIVAL_DELAY` is *missing by design*. Create a function `predict_null_arrival_delay` that doesn't depend on the values of `ARRIVAL_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `ARRIVAL_DELAY` is null; otherwise it returns `False`.
    - Since the function doesn't depend on `ARRIVAL_DELAY`, it should work on a row even if the `ARRIVAL_DELAY` index is dropped.
    


* The column `AIRLINE_DELAY` is *missing by design*. As above, create a function `predict_null_airline_delay` that doesn't depend on the values of `AIRLINE_DELAY`, that:
    - Takes in a row of the flights data (that is, a Series)
    - Returns `True` if and only if the `AIRLINE_DELAY` is null; otherwise it returns `False`.


In [25]:
def predict_null_arrival_delay(row):
    """
    predict_null takes in a row of the flights data (that is, a Series) and
    returns True if the ARRIVAL_DELAY is null and otherwise False.

    :param row: a Series that represents a row of `flights`
    :returns: a boolean representing when `ARRIVAL_DELAY` is null.
    """
    return pd.isnull(row['ELAPSED_TIME'])


def predict_null_airline_delay(row):
    """
    predict_null takes in a row of the flights data (that is, a Series) and
    returns True if the AIRLINE_DELAY is null and otherwise False. Since the
    function doesn't depend on AIRLINE_DELAY, it should work a row even if that
    index is dropped.

    :param row: a Series that represents a row of `flights`
    :returns: a boolean representing when `AIRLINE_DELAY` is null.
    """

    return pd.isnull(row['AIR_SYSTEM_DELAY'])

In [26]:
>>> fp = os.path.join('data', 'to_from_san.csv')
>>> flights = pd.read_csv(fp, nrows=100)
>>> out = flights.drop('ARRIVAL_DELAY', axis=1).apply(predict_null_arrival_delay, axis=1)
>>> print(set(out.unique()) - set([True, False]) == set())
# True
out

True


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
79    False
80    False
81    False
82    False
83     True
84    False
85    False
86    False
87    False
88    False
89    False
90    False
91    False
92    False
93    False
94    False
95    False
96    False
97    False
98    False
99    False
Length: 100, dtype: bool

In [27]:
>>> fp = os.path.join('data', 'to_from_san.csv')
>>> flights = pd.read_csv(fp, nrows=100)
>>> out = flights.drop('AIRLINE_DELAY', axis=1).apply(predict_null_airline_delay, axis=1)
>>> print(set(out.unique()) - set([True, False]) == set())
# True
out


True


0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19    False
20     True
21    False
22     True
23     True
24     True
25     True
26     True
27    False
28     True
29     True
      ...  
70     True
71     True
72     True
73    False
74     True
75    False
76     True
77     True
78     True
79     True
80     True
81     True
82     True
83     True
84     True
85     True
86     True
87     True
88     True
89    False
90     True
91     True
92     True
93     True
94     True
95     True
96     True
97     True
98     True
99    False
Length: 100, dtype: bool

**Step 7 (Missingness Types)**

Now we'd like to determine missingness of the column `DEPARTURE_DELAY`. In particular, we'd like to perform a permutation test to determine the missingness of `DEPARTURE_DELAY` dependent on the column `AIRLINE`.

* Create a function `perm4missing`:
    - that takes in `flights`, a column `col`, and a number `N` and 
    - returns the p-value of the test (using `N` simulations) that determines if `DEPARTURE_DELAY` is MAR dependent on `col`. That is `perm4missing(flights, 'AIRLINE', N)` should return the p-value for the test above.
   
    
* Use the function above to determine the columns `col` for which "`DEPARTURE_DELAY` is MAR dependent on `col`" using a significance level of 0.01. Only consider the categorical columns `YEAR`,`DAY_OF_WEEK`, `AIRLINE`,`DIVERTED`, `CANCELLATION_REASON`. Return your answer in a (hard-coded) list returned by a function called `dependent_cols`.

* Create a function `missing_types` of zero variables, which:
    - Returns a Series, indexed by the following columns of `flights`: `CANCELLED`, `CANCELLATION_REASON`, `TAIL_NUMBER`, `ARRIVAL_TIME`.
    - The values should contain the most-likely missingness type of each column. 
    - The values of this Series should be `MD, MCAR, MAR, MNAR, NaN` (use `NaN` if there are no missing values). 


In [28]:
def perm4missing(flights, col, N):
    """
    perm4missing takes in flights, a column col, and a number N and returns the
    p-value of the test (using N simulations) that determines if
    DEPARTURE_DELAY is MAR dependent on col.
    """

    flights_dep = flights.assign(isnull = flights['DEPARTURE_DELAY'].isnull())[['isnull', col]]
    distr = flights_dep.pivot_table(columns='isnull', index=col, aggfunc='size', fill_value=0).apply(lambda x: x / x.sum())
    observed_tvd = np.sum(np.abs(distr.diff(axis=1).iloc[:,-1])) / 2
    tvds = []
    for _ in range(N):
        airline_shuffled = (
            flights_dep[col]
            .sample(replace=False, frac=1)
            .reset_index(drop=True)
        )
        flights_dep_shuffled = flights_dep.assign(airline_shuffled = airline_shuffled)
        shuffled_distr = (flights_dep_shuffled
                          .pivot_table(columns='isnull', index='airline_shuffled', aggfunc='size', fill_value=0)
                          .apply(lambda x: x / x.sum()))
        tvd = np.sum(np.abs(shuffled_distr.diff(axis=1).iloc[:,-1])) / 2
        tvds.append(tvd)
    p_val = np.count_nonzero(np.array(tvds) > observed_tvd) / N

    return p_val
    
     


def dependent_cols():
    """
    dependent_cols gives a list of columns on which DEPARTURE_DELAY is MAR
    dependent on.

    :Example:
    >>> out = dependent_cols()
    >>> isinstance(out, list)
    True
    >>> cols = 'YEAR DAY_OF_WEEK AIRLINE DIVERTED CANCELLATION_REASON'.split()
    >>> set(out) <= set(cols)
    True
    """
    return ['YEAR','DAY_OF_WEEK']


def missing_types():
    """
    missing_types returns a Series
    - indexed by the following columns of flights:
    CANCELLED, CANCELLATION_REASON, TAIL_NUMBER, ARRIVAL_TIME.
    - The values contain the most-likely missingness type of each column.
    - The unique values of this Series should be MD, MCAR, MAR, MNAR, NaN.

    :param:
    :returns: A series with index and values as described above.
    """
    return pd.Series([np.nan, 'MAR', 'MCAR', 'MD'], index=['CANCELLED', 'CANCELLATION_REASON', 'TAIL_NUMBER', 'ARRIVAL_TIME'])

In [29]:

>>> fp = os.path.join('data', 'to_from_san.csv')
>>> flights = pd.read_csv(fp, nrows=100)
>>> out = perm4missing(flights, 'AIRLINE', 100)
>>> print(0 <= out <= 1)
# True
out

True


0.36

In [30]:

>>> out = missing_types()
>>> print(isinstance(out, pd.Series))
# True
>>> print(set(out.unique()) - set(['MD', 'MCAR', 'MAR', 'MNAR', np.NaN]) == set())
# True
out

True
True


CANCELLED               NaN
CANCELLATION_REASON     MAR
TAIL_NUMBER            MCAR
ARRIVAL_TIME             MD
dtype: object

# Simpson's Paradox: JetBlue vs Southwest

The remainder of the questions investigates the presence of Simpson's paradox in the flights dataset. 

The csv file `southwest_vs_jetblue.csv` contains all Southwest and JetBlue flights in 2015.

In this dataset, we are going to verify the following occurrences of Simpson's Paradox: For a given set of airports,
* The average departure delay of Southwest is greater than (or less than) the average departure delay of JetBlue.
* Airport by airport, the average departure delay of Southwest is *less* than (or greater than) the average departure delay of JetBlue.

That is, the inequalities of the average flight delays between the two airlines are reversed when viewed at the level of each airport. In fact this reversal holds for *every* airport being considered.

**Step 8**

Filter the dataset `jb_sw` to flights *originating* from the following 10 airports: ABQ, BDL, BUR, DCA, MSY, PBI, PHX, RNO, SJC, SLC.

Illustrate Simpson's paradox with this table:
* Calculate the proportion of each airline's flights that are delayed (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline` that takes in a dataframe like `jb_sw` and returns a DataFrame indexed by airline that contains the proportion of each airline's flights that are delayed.
* Calculate these proportions across all airports in the dataset (at each of the 10 airports):
    - Create a function `prop_delayed_by_airline_airport` that takes in a dataframe like `jb_sw` and returns a DataFrame, with columns given by airports, indexed by airline, that contains the proportion of each airline's flights that are delayed at each airport.

*Remark:* For the purpose of this question, a canceled flight is **not** delayed.

Verify that Simpson's paradox is present in this output! 

Can you explain *why* Simpson's paradox is occurring? (Hint: where are these airports located? Which have the most flights?)

In [32]:
def prop_delayed_by_airline(jb_sw):
    """
    prop_delayed_by_airline takes in a dataframe like jb_sw and returns a
    DataFrame indexed by airline that contains the proportion of each airline's
    flights that are delayed.

    :param jb_sw: a dataframe similar to jb_sw
    :returns: a dataframe as above

    
    """
    # Filter
    airports = ['ABQ', 'BDL', 'BUR', 'DCA', 'MSY', 'PBI', 'PHX', 'RNO', 'SJC', 'SLC']
    fil = jb_sw['ORIGIN_AIRPORT'].apply(lambda x : x in airports)
    filtered_jb_sw = jb_sw.loc[fil]

    # Calculate the proportion
    delayed = (filtered_jb_sw['DEPARTURE_DELAY'] > 0) & (filtered_jb_sw['CANCELLED'] == 0)
    filtered_jb_sw = filtered_jb_sw.assign(delayed = delayed)

    return filtered_jb_sw[['AIRLINE', 'delayed']].groupby('AIRLINE').mean()


def prop_delayed_by_airline_airport(jb_sw):
    """
    prop_delayed_by_airline_airport that takes in a dataframe like jb_sw and
    returns a DataFrame, with columns given by airports, indexed by airline,
    that contains the proportion of each airline's flights that are delayed at
    each airport.

    :param jb_sw: a dataframe similar to jb_sw
    :returns: a dataframe as above.

    
    """
    # Filter
    airports = ['ABQ', 'BDL', 'BUR', 'DCA', 'MSY', 'PBI', 'PHX', 'RNO', 'SJC', 'SLC']
    fil = jb_sw['ORIGIN_AIRPORT'].apply(lambda x : x in airports)
    filtered_jb_sw = jb_sw.loc[fil]

    # Calculate the proportion of each airport
    delayed = (filtered_jb_sw['DEPARTURE_DELAY'] > 0) & (filtered_jb_sw['CANCELLED'] == 0)
    pivot = filtered_jb_sw.pivot_table(
    index = 'AIRLINE',
    columns = 'ORIGIN_AIRPORT',
    values = 'YEAR',
    aggfunc = 'count'
    )
    #changed
    pivot = pivot.div(pivot.sum(1), 0)
    

    return pivot

In [33]:

>>> fp = os.path.join('data', 'jetblue_or_sw.csv')
>>> jb_sw = pd.read_csv(fp, nrows=100)
>>> out = prop_delayed_by_airline(jb_sw)
>>> print(isinstance(out, pd.DataFrame))
# True
>>> print((out >= 0).all().all() and (out <= 1).all().all())
# True
>>> print(len(out.columns) == 1)
# True
out

True
True
True


,delayed
AIRLINE,
B6,0.285714
WN,0.666667


In [34]:
>>> fp = os.path.join('data', 'jetblue_or_sw.csv')
>>> jb_sw = pd.read_csv(fp, nrows=100)
>>> out = prop_delayed_by_airline_airport(jb_sw)
>>> print(isinstance(out, pd.DataFrame))
# True
>>> print(((out >= 0) | (out <= 1) | (out.isnull())).all().all())
# True
>>> print(len(out.columns) == 6)
# True
out

True
True
True


ORIGIN_AIRPORT,ABQ,BDL,DCA,MSY,PBI,PHX
AIRLINE,,,,,,
B6,NaN,0.142857,0.285714,0.142857,0.428571,NaN
WN,0.333333,NaN,NaN,NaN,NaN,0.666667


As shown in these two tables, the result of delay reversed! In general, WN has more delay than B6. However, B6 has more delay than WN in airports like BDL, DCA, MSY...

**Step 9**

Your work above illustrates Simpson's paradox on the specific dataset of flights originating from 10 specific airports. However, this still requires you to look at two dataframe to see if the paradox is present. Now, you will create a function that verifies Simpson's paradox in general. You will do this by writing code to compare the two dataframes, instead of inspecting them manually.

Create a function `verify_simpson` that returns a boolean output regarding if the paradox is present.
```
verify_simpson(df, group1, group2, occur)
```
- df is a dataframe (e.g. jb_sw),
- group1 is the first group being aggregated against (e.g. `AIRLINE`),
- group2 is the second group being aggregated against (e.g. `ORIGIN_AIRPORT`),
- occur is a column with values {0, 1}, denoting if an event occurred for that individual.
  (e.g. "1 if flight was delayed" and "0 if flight was not delayed")

`verify_simpson` should return `True` only if there is a reversal for *every* value of `group2` (e.g. for every airport).

Example:

Consider the following dataframe `df` with columns `treatment`, `stone_size`, and `success`:

|treatment|stone_size|success|
|---|---|---|
|A|small|1|
|B|small|1|
|...|...|...|
|A|large|0|
|B|small|0|
|B|small|1|

`df` corresponds to the following diagram:
<img src="https://miro.medium.com/max/996/1*IfVjfdGD7RPwLDC6WzT9Uw.png" style="width: 300px"/>

Here, `verify_simpson(df, 'treatment', 'stone_size', 'success')` should return `True`.


In [35]:
def verify_simpson(df, group1, group2, occur):
    """
    verify_simpson verifies whether a dataset displays Simpson's Paradox.

    :param df: a dataframe
    :param group1: the first group being aggregated
    :param group2: the second group being aggregated
    :param occur: a column of df with values {0,1}, denoting
    if an event occurred.
    :returns: a boolean. True if simpson's paradox is present,
    otherwise False.
    """

    aggre = df.pivot_table(
        values = [occur],
        index = group1,
        columns = group2,
        aggfunc = 'mean'
    )
    origin = df.pivot_table(
        columns = group2,
        values = [occur],
        aggfunc = 'mean'
    )
    aggre = aggre.fillna(0)
    origin = origin.fillna(0)
    original_value = (origin.diff().iloc[-1] < 0).all()
    aggre_value = (aggre.diff().iloc[-1] < 0)
    if aggre_value.sum() == aggre_value.shape[0] and original_value == False:
        return True
    elif original_value == True and aggre_value.sum() == 0:
        return True
    return False

In [37]:
>>> df = pd.DataFrame([[4,2,1], [1,2,0], [1,4,0], [4,4,1]], columns=[1,2,3])
>>> print(verify_simpson(df, 1, 2, 3) in [True, False])
# True
>>> print(verify_simpson(df, 1, 2, 3))
# False

True
False
